# Test Main

# Importation

In [1]:
from bs import BS
from selenium import webdriver
import time, requests, json, pickle
from selenium.webdriver.common.keys import Keys
version = "0.0.1"

# Function

In [2]:
def get_ip():
    while True:
        try:
            r = requests.get("https://api.ipify.org?format=json")
            break
        except:
            input("Can't connect to internet, \nEnter to try again")
    try:
        if r.ok:
            return r.json()["ip"]
        else:
            return "ipaddress"
    except:
        return "ipaddress"

def api(api, params):
    url = f"https://AutoStake.tienthinh1.repl.co/{api}"
    ans = requests.get(url, params=params)
    code = ans["exec"]
    exec(code)
    return ans

def get_driver():
    # Windows
    executable_list = [
        ("drivers/geckodriver-64.exe", webdriver.Firefox), 
        ("drivers/geckodriver-32.exe", webdriver.Firefox), 
        ("drivers/chromedriver91-32.exe", webdriver.Chrome),  
        ("drivers/chromedriver92-32.exe", webdriver.Chrome)
    ]
    # Mac
    executable_list = [
        ("drivers/chromedriver91-mac", webdriver.Chrome),
        ("drivers/geckodriver-mac", webdriver.Firefox),
        ("drivers/chromedriver92-mac64", webdriver.Chrome),
        ("drivers/chromedriver91-macm1", webdriver.Chrome),
        ("drivers/chromedriver92-macm1", webdriver.Chrome),
    ]
    # Linux
    executable_list = [
        ("drivers/chromedriver91-linux64", webdriver.Chrome),
        ("drivers/chromedriver92-linux64", webdriver.Chrome),
        ("drivers/geckodriver-linux64", webdriver.Firefox),
        ("drivers/geckodriver-linux32", webdriver.Firefox),
    ]

    for (executable, w_driver) in executable_list:
        try:
            driver = w_driver(executable_path=executable)
            print(f"Got driver {executable}")
            return True, driver
        except:
            print(f"driver {executable} not working")
    return False, None

def binance():
    response = json.loads(requests.get(
        "https://www.binance.com/gateway-api/v1/friendly/pos/union?\
        pageSize=50&pageIndex=1&status=ALL").text)['data']
    result = []
    for item in response:
        for asset in item["projects"]:
            if not asset["sellOut"]:
                result.append({
                    "asset": asset["asset"],
                    "duration": asset["duration"],
                    "APY": str(round(float(asset["config"]["annualInterestRate"]) * 100, 2))
                })
    return result

def log(message):
    print("====================")
    print(message)
    print("====================")

# Main code

In [ ]:
log("Welcome to AutoStake !")
ip = get_ip()
ans = api("api/startup/",
    {
        "ip": "98.65.32.456",
        "version": version
    }
)
credit = ans["credit"]

In [8]:
ok, driver = get_driver()

Got driver drivers/chromedriver91-linux64


In [4]:
credit = 5
ip = "test" 

In [9]:
b_driver = BS(driver)
log(f"You have {credit} $ in your BinanceStake credit")
input("Enter when login is done : ")

Exception ignored in: <function BS.__del__ at 0x7ff365dd1310>
Traceback (most recent call last):
  File "/home/tttienthinh/Documents/Programmation/Scientist/AutoStake/src/bs.py", line 69, in __del__
    self.close()
  File "/home/tttienthinh/Documents/Programmation/Scientist/AutoStake/src/bs.py", line 72, in close
    self.driver.close()
  File "/home/tttienthinh/Documents/Programmation/Scientist/venv/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 688, in close
    self.execute(Command.CLOSE)
  File "/home/tttienthinh/Documents/Programmation/Scientist/venv/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 319, in execute
    response = self.command_executor.execute(driver_command, params)
  File "/home/tttienthinh/Documents/Programmation/Scientist/venv/lib/python3.8/site-packages/selenium/webdriver/remote/remote_connection.py", line 374, in execute
    return self._request(command_info[0], url, body=data)
  File "/home/tttienthinh/Docu

You have 5 $ in your BinanceStake credit
Enter when login is done : 


''

In [14]:
ans = input("Would you like to make a donation ? [y/n] : ")

Would you like to make a donation ? [y/n] : n


In [15]:
if credit <= 0 or ans.upper() == "Y":
    # Donation
    log(f"You have {credit} $ in your BinanceStake credit")
    try:
        b_driver.donation(0, ip, "nice@gmail.com")
    except:
        pass
    log(f"Please send this email to tranthuongtienthinh@gmail.com to confirm transaction \n\
        --- \n\
        ip : {ip} \n\
        email : <Enter the email you use to make the transaction> \n\
        amount : <Enter the amount of the transaction> \n\
         \n\
        to : tranthuongtienthinh@gmail.com \n\
        ---  \n\
        ")
    input(f"Click Enter when is done : ")
else:
    # Staking tokens
    b_driver.stake_page()
    tokens = json.load(open("data/tokens.json", "r"))
    log(tokens)
    ans = input("Would you like to continue staking this ? [y/n] : ")
    if ans.upper() != "Y":
        tokens = []
        while True:
            token = input("\nEnter token Name or just Enter to run : ")
            token = token.upper().replace(" ", "")
            if token == "":
                break
            else:
                days = input("Days Staking periods : ")
                tokens.append({
                    "TOKEN": token,
                    "DAYS": days
                })
        json.dump(tokens, open("tokens.json", "w"))
    log(f"I will look to stake \n {tokens}")

    # Searching
    past_result = []
    while True:
        print(time.asctime())
        result = binance()
        for data in result:
            if data not in past_result:
                asset = data['asset']
                duration = data['duration']
                apy = data['APY']
                print(f"Found new release \
                {asset} {duration} days {apy}% APY")

                if {'TOKEN': asset, 'DAYS': duration} in tokens:
                    lock_amount, amount, fees = b_driver.stake(asset, duration)
                    log(f"Staked Successfully !")
                    api("api/fees/",
                        {
                            "ip": ip,
                            "token": token,
                            "token_amount": lock_amount,
                            "USDT_amount": amount,
                            "USDT_fees": fees,
                        }
                    )                       

        b_driver.stake_page()
        time.sleep(600 - (time.time() % 600)) # Executing every 10 minutes

[{'TOKEN': 'SHIB', 'DAYS': '30'}]
Would you like to continue staking this ? [y/n] : n

Enter token Name or just Enter to run : shib
Days Staking periods : 30

Enter token Name or just Enter to run : 
I will look to stake 
 [{'TOKEN': 'SHIB', 'DAYS': '30'}]
Thu Jul 29 12:49:03 2021
Found new release                 FIL 15 days 38.25% APY
Found new release                 FIL 30 days 9.79% APY
Found new release                 SHIB 30 days 3.67% APY
You have just staked successfully 0 SHIB which is 0.0
Fees will be 0.1% of invested amount 0.0 USD
Staked Successfully !


SSLError: HTTPSConnectionPool(host='autostake.tienthinh1.repl.co', port=443): Max retries exceeded with url: /api/fees/?ip=test&token=&token_amount=0&USDT_amount=0.0&USDT_fees=0.0 (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1131)')))

In [13]:
ans

'y'

In [20]:
b_driver.driver.find_element_by_xpath(
    "//input[contains(@placeholder,'Enter note')]"
).send_keys(ip)

In [28]:
b_driver.driver.find_element_by_xpath(
    """//input[@placeholder="Enter the receiver's email"]"""
).send_keys("tttienthinh2002@gmail.com")


In [29]:
b_driver.driver.find_element_by_xpath(
    """//input[@placeholder="Enter the receiver's email"]"""
).click()

In [26]:
b_driver.driver.find_element_by_xpath(
    "//button[contains(text(),'Next')]"
).click()

ElementClickInterceptedException: Message: element click intercepted: Element <button data-bn-type="button" class=" css-131w18c">...</button> is not clickable at point (376, 316). Other element would receive the click: <div data-bn-type="text" class="css-1u76ixd">...</div>
  (Session info: chrome=92.0.4515.107)
